In [1]:
import pandas as pd
import tarfile
import json
import polars as pl

In [11]:
#Buisness

# Define the JSON file path
json_file_path = "yelp_academic_dataset_business.json"

# Read the JSON file in batches
buinsess = pl.scan_ndjson(json_file_path)

# Select only the 'user_id' and 'text' columns
buinsess_subset = buinsess.select(['business_id', 'name']).collect(streaming = True)

(150346, 2)

In [12]:
#Users

# Define the JSON file path
json_file_path = "yelp_academic_dataset_user.json"

# Read the JSON file in batches
users = pl.scan_ndjson(json_file_path)

# Select only the 'user_id' and 'text' columns
users_subset = users.select(['user_id', 'elite']).collect(streaming = True)

In [81]:
#Reviews

# Define the JSON file path
json_file_path = "yelp_academic_dataset_review.json"

# Read the JSON file in batches
reviews = pl.scan_ndjson(json_file_path)

# Select only the 'user_id' and 'text' columns
reviews_subset = reviews.select(['user_id', 'text', 'date', 'business_id', 'stars', 'useful', 'funny', 'cool']).collect(streaming = True)


In [84]:
stars_counts = reviews_subset['stars'].value_counts()

In [85]:
stars_counts

stars,counts
f64,u32
1.0,1069561
2.0,544240
3.0,691934
4.0,1452918
5.0,3231627


In [ ]:
# Define the fraction of rows to sample
frac = 0.005  # For example, to sample 10% of the rows

# Define the column(s) to control the sample
control_columns = ["stars", "useful", "funny", "cool"]  # Replace with your desired columns

# Group by the control columns
grouped_df = reviews_subset.groupby(control_columns)

# Convert grouped DataFrame to a list of DataFrames
grouped_dfs = [group for group in grouped_df]

# Sample each DataFrame individually
sampled_dfs = [group.sample(fraction=frac) for group in grouped_dfs]

# Concatenate the sampled DataFrames back together
sampled_df = pl.concat(sampled_dfs, axis=0)

In [14]:
reviews_users_df = reviews_subset.join(users_subset, on='user_id', how='inner')

In [20]:
elite_users_df = reviews_users_df.filter(pl.col('elite') != "")

In [52]:
elite_users_df.head()

user_id,text,date,business_id,elite
str,str,str,str,str
"""8g_iMtfSiwikVn…","""Family diner. …","""2014-02-05 20:…","""YjUWPpI6HXG530…","""2014,2015,2016…"
"""9OAtfnWag-ajVx…","""Love going her…","""2014-06-27 22:…","""lj-E32x9_FA7Gm…","""2014,2015,2016…"
"""4Uh27DgGzsp6Pq…","""The bun makes …","""2011-10-27 17:…","""otQS34_MymijPT…","""2011,2012"""
"""j2wlzrntrbKwyO…","""The hubby and …","""2014-08-10 19:…","""rBdG_23USc7Dle…","""2011,2012,2013…"
"""IQsF3Rc6IgCzjV…","""My absolute fa…","""2014-11-12 15:…","""eFvzHawVJofxSn…","""2018"""


In [54]:
elite_users_df = elite_users_df.select(
    "*",
    pl.col("date").apply(lambda date: date.split("-")[0]).alias("comment_year")
)

In [56]:
filtered_data = elite_users_df.filter(
    pl.col("comment_year").is_in(pl.col("elite"))
)

In [60]:
filtered_data = filtered_data.filter(
    pl.col("comment_year").is_in(pl.col("elite").str.split(",").explode())
)

In [63]:
filtered_data = elite_users_df.filter(
    pl.col("elite").str.contains(pl.col("comment_year").cast(pl.Utf8()))
)

In [67]:
reviews_buisness = filtered_data.join(buinsess_subset, on='business_id', how='inner')

In [73]:
reviews_buisness = reviews_buisness.select(
    "*", 
    pl.col('text').str.lengths().alias('text_length')
)


In [76]:
yelp_verified = reviews_buisness.select(pl.col(["text","date", "name", "text_length"]))

In [78]:
yelp_verified = yelp_verified.rename({"name": "business_name"})

In [80]:
# Define the file path
path = "./yelp_verified.csv"

# Write the DataFrame to a CSV file
yelp_verified.write_csv(path, separator=",")